# Data Extraction for my MVP:

In [59]:
import pandas as pd
import numpy as np
import matplotlib as mtpl

- First we ingest all the csv.

In [60]:
ucluj_dataset_mvp = pd.read_csv("../../datasets/Universitatea_Cluj_2024_2025_events.csv")

- Then we will try to extract the required columns from U Cluj Dataset provided as CSV from DataCamp Official. 

In [61]:
filtered_ucluj_dataset_mvp = ucluj_dataset_mvp[["id","matchId","matchPeriod","minute","second","type.primary","type.secondary","location.x","location.y","team.id","team.name","opponentTeam.id","opponentTeam.name","player.id","player.name","pass.endLocation.x","pass.endLocation.y","shot.xg","shot.isGoal","shot.onTarget","carry.endLocation.x","carry.endLocation.y","possession.id","possession.duration","possession.startLocation.x","possession.startLocation.y","possession.endLocation.x","possession.endLocation.y","competitionId","seasonId","Home_Away"
]]
filtered_ucluj_dataset_mvp.head()

,id,matchId,matchPeriod,minute,second,type.primary,type.secondary,location.x,location.y,team.id,...,carry.endLocation.y,possession.id,possession.duration,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,competitionId,seasonId,Home_Away
0,2555084360,5695562,1H,0,1,pass,"['back_pass', 'short_or_medium_pass']",49.0,50.0,11566,...,NaN,2.555084e+09,29.639864,49.0,50.0,46.0,3.0,719,190857,Home
1,2555084372,5695562,1H,0,3,pass,"['forward_pass', 'long_pass', 'loss', 'pass_to...",34.0,65.0,11566,...,NaN,2.555084e+09,29.639864,49.0,50.0,46.0,3.0,719,190857,Home
2,2555084324,5695562,1H,0,9,pass,"['loss', 'progressive_pass', 'short_or_medium_...",14.0,89.0,60374,...,NaN,2.555084e+09,29.639864,49.0,50.0,46.0,3.0,719,190857,Home
3,2555084377,5695562,1H,0,9,interception,"['recovery', 'counterpressing_recovery']",84.0,18.0,11566,...,NaN,2.555084e+09,29.639864,49.0,50.0,46.0,3.0,719,190857,Home
4,2555084325,5695562,1H,0,11,pass,"['forward_pass', 'progressive_pass', 'short_or...",26.0,89.0,60374,...,NaN,2.555084e+09,29.639864,49.0,50.0,46.0,3.0,719,190857,Home


## Build the conceptual Grid

- First we check to see how many games are included in our datasets just to be sure

In [62]:
filtered_ucluj_dataset_mvp.groupby(["team.id","Home_Away"]).ngroups

32

- we select all the passes and carries from the opponents of ucluj from last season.

In [63]:
df_passes_and_carries_ucluj = filtered_ucluj_dataset_mvp[
    ((filtered_ucluj_dataset_mvp["type.primary"]=="pass")|
    (filtered_ucluj_dataset_mvp["type.primary"]=="carry"))&
    (filtered_ucluj_dataset_mvp["team.id"]!=60374)]

- now we need to create the function for adding to our existing dataframe the column with the specific location on our pitch grid plotting 

In [64]:
#pitch grid:8x6
def calculating_pitch_zone_assignation(x,y):
    row = x//12.5
    column = y//16.67
    result = "zone_"+str(int(row))+"_"+str(int(column))
    return result


In [ ]:
df_passes_and_carries_ucluj["start_zone"] = df_passes_and_carries_ucluj.apply(
    lambda row: calculating_pitch_zone_assignation(row["location.x"], row["location.y"]),
    axis=1
)
df_passes_and_carries_ucluj["end_zone"] = df_passes_and_carries_ucluj.apply(
    lambda row: calculating_pitch_zone_assignation(row["pass.endLocation.x"], row["pass.endLocation.y"]),
    axis=1
)
df_passes_and_carries_ucluj[["location.x","location.y","pass.endLocation.x","pass.endLocation.y"]].head()

C:\Users\bodoz\AppData\Local\Temp\ipykernel_5004\611509255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes_and_carries_ucluj["start_zone"] = df_passes_and_carries_ucluj.apply(
C:\Users\bodoz\AppData\Local\Temp\ipykernel_5004\611509255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes_and_carries_ucluj["end_zone"] = df_passes_and_carries_ucluj.apply(


,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,49.0,50.0,34.0,65.0
1,34.0,65.0,86.0,11.0
5,56.0,11.0,33.0,22.0
6,33.0,22.0,11.0,42.0
7,11.0,42.0,21.0,14.0


In [96]:
df_frequncy_passes = df_passes_and_carries_ucluj.groupby(["start_zone","end_zone"]).size().reset_index(name="frequency")
df_frequncy_passes_total_end_zones = df_frequncy_passes.groupby(["end_zone"]).size().reset_index(name="frequencytest")

df_frequncy_passes

,start_zone,end_zone,frequency
0,zone_0_0,zone_0_0,16
1,zone_0_0,zone_0_1,4
2,zone_0_0,zone_0_2,3
3,zone_0_0,zone_1_0,18
4,zone_0_0,zone_1_1,9
...,...,...,...
1367,zone_7_5,zone_7_5,11
1368,zone_7_5,zone_8_1,2
1369,zone_7_5,zone_8_5,14
1370,zone_8_1,zone_7_3,1
